# **Intro data analysis**

**i.** Import libraries

In [ ]:
import pandas as pd
import numpy as np
import sklearn
import lightgbm
import scipy
import statsmodels
import matplotlib
import seaborn

**ii.** Load data from kaggle with pandas.

**iii.** What is the size of your data?

**iv.** Print the list of columns. Which column is a target?

**v.** Make a fast analysis of the data: use methods info(), describe(), corr(). Explain the results of outputs. Are there any empty columns?

**vi.** We’ll work only with 3 features: 'bathrooms', 'bedrooms', 'interest_level' and with target column ‘price’. Make a dataframe with these columns only.

# Statistical data analysis

**i.** To start with statistical data analysis we recommend you refresh basic knowledge of statistics, such as Mean / Median / Mode / Variance / Standard Deviation. Also you are welcome to be free with distributions (Discrete uniform Distribution, Bernoulli Distribution, Binomial Distribution, Poisson Distribution, Normal Distribution, Exponential Distribution). Please make sure that you know the definitions of outliers, percentiles, confidential intervals.

**ii.** Make a quick sense with this article. Please take attention to such aspects as distributions and histograms, boxplot, outliers, kernel density function.

**iii.** Target analysis

**a.** Plot a histogram to understand the distribution of the target. Is it all clear?

**b.** The next step is boxplot(). What can you say about target? Are there any outliers?

**c.** Drop rows which are out of the 1 and 99 percentile by the target column.

**d.** Plot a histogram for price again. Explain the result.